In [9]:
import os

# Specify the directory where your CSV files are located
directory = r'D:\github\Cricket-Prediction\data\1_rawData' 

# sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CricketPrediction").getOrCreate()

team_data = spark.read.csv(os.path.join(directory, 't20_team_stats.csv'), header=True, inferSchema=True)
team_data.show()

+--------+---+---+----+----+---+-----+-----+-----+----+---+---+-------+
|    Team|Mat|Won|Lost|Tied| NR|  W/L|  Ave|  RPO|Inns| HS| LS| Season|
+--------+---+---+----+----+---+-----+-----+-----+----+---+---+-------+
|Zimbabwe|  5|  5|   0|   0|  0|    -| 43.6|12.82|   5|344|  -|2024/25|
|Zimbabwe| 10|  2|   8|   0|  0|0.250|17.48| 7.16|  10|159|124|   2024|
|Zimbabwe| 17|  8|   9|   0|  0|0.888|24.03| 7.81|  17|217| 82|2023/24|
|Zimbabwe| 11|  5|   5|   0|  1|1.000|16.94| 7.13|  11|174|115|2022/23|
|Zimbabwe| 16|  9|   7|   0|  0|1.285|22.45| 7.75|  16|236| 95|   2022|
|Zimbabwe| 14|  6|   8|   0|  0|0.750| 20.6| 7.22|  14|193|138|   2021|
|Zimbabwe|  6|  0|   6|   0|  0|0.000|19.38| 7.28|   6|156|148|2020/21|
|Zimbabwe|  6|  3|   3|   0|  0|1.000|24.56| 7.98|   6|177|152|2019/20|
|Zimbabwe|  8|  2|   5|   1|  0|0.400|22.01| 8.23|   8|172|136|   2019|
|Zimbabwe|  2|  0|   2|   0|  0|0.000|15.17| 6.91|   2|132|126|2018/19|
|Zimbabwe|  4|  0|   4|   0|  0|0.000|17.74| 7.06|   4|162|108| 

In [ ]:
from pyspark.sql.functions import col,when
team_data = team_data.withColumn("W/L", when(col("Lost")==0, col("Won")).otherwise(col("Won")/col("Lost")))
team_data = team_data.withColumn("LS", when(col("LS")=='-',0).otherwise(col("LS")).cast("int"))
team_data = team_data.withColumn("AveRPW", when(col("Ave")=='-',0).otherwise(col("Ave")).cast("float")).drop("Ave")
team_data = team_data.withColumn("AveRPO", when(col("RPO")=='-',0).otherwise(col("RPO")).cast("float")).drop("RPO")
team_data.show()

+--------+---+---+----+----+---+------------------+----+---+---+-------+------+------+
|    Team|Mat|Won|Lost|Tied| NR|               W/L|Inns| HS| LS| Season|AveRPW|AveRPO|
+--------+---+---+----+----+---+------------------+----+---+---+-------+------+------+
|Zimbabwe|  5|  5|   0|   0|  0|               5.0|   5|344|  0|2024/25|  43.6| 12.82|
|Zimbabwe| 10|  2|   8|   0|  0|              0.25|  10|159|124|   2024| 17.48|  7.16|
|Zimbabwe| 17|  8|   9|   0|  0|0.8888888888888888|  17|217| 82|2023/24| 24.03|  7.81|
|Zimbabwe| 11|  5|   5|   0|  1|               1.0|  11|174|115|2022/23| 16.94|  7.13|
|Zimbabwe| 16|  9|   7|   0|  0|1.2857142857142858|  16|236| 95|   2022| 22.45|  7.75|
|Zimbabwe| 14|  6|   8|   0|  0|              0.75|  14|193|138|   2021|  20.6|  7.22|
|Zimbabwe|  6|  0|   6|   0|  0|               0.0|   6|156|148|2020/21| 19.38|  7.28|
|Zimbabwe|  6|  3|   3|   0|  0|               1.0|   6|177|152|2019/20| 24.56|  7.98|
|Zimbabwe|  8|  2|   5|   1|  0|           

In [12]:
team_data.toPandas().to_csv(r'D:\github\Cricket-Prediction\data\2_processedData\teamStats.csv', index=False)